In [84]:
import pandas as pd

quota = pd.read_html('quota.html', match= "Quota")

In [85]:
print(len(quota))

1


In [86]:
df = quota[0]
df.head()

,0,1,2,3,4
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,Department,Statu,Quota,Current,NaN
2,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71,NaN
3,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1,NaN


In [87]:
df.columns = ['Department', 'Status', "Quota", "Current","a"]
df.head()

,Department,Status,Quota,Current,a
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,Department,Statu,Quota,Current,NaN
2,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71,NaN
3,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1,NaN


In [88]:
df.drop("a",axis=1, inplace=True)
df

,Department,Status,Quota,Current
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,Department,Statu,Quota,Current
2,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71
3,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1


In [89]:
df.drop(labels=range(0,2), inplace=True)
df = df.reset_index(drop=True)
df

,Department,Status,Quota,Current
0,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71
1,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1


In [90]:
# import requests
# from bs4 import BeautifulSoup
 
 
# url = 'https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=MIS&bolum=MANAGEMENT+INFORMATION+SYSTEMS'
# reqs = requests.get(url)
# soup = BeautifulSoup(reqs.text, 'html.parser')
 
# urls = []
# for link in soup.find_all('a'):
#     print(link.get('href'))

In [91]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np

driver = webdriver.Chrome()
driver.get("https://registration.boun.edu.tr/scripts/sch.asp?donem=2022/2023-2&kisaadi=MIS&bolum=MANAGEMENT+INFORMATION+SYSTEMS")

info_links = driver.find_elements(By.LINK_TEXT,"Info")

dfs = []

for info_link in info_links:
    info_link.click()
    # Switch to the new window
    
    new_window = driver.window_handles[-1]
    driver.switch_to.window(new_window)
    
    # Find the table element
    table = driver.find_element(By.TAG_NAME, "table")
    
    # Convert the table content to a DataFrame
    df2 = pd.read_html(table.get_attribute('outerHTML'))[0]
    
    strongs = driver.find_elements(By.TAG_NAME, "strong")
    strongs_text = [strong.text for strong in strongs]
    
    strongs_text_repeated = np.repeat(strongs_text, len(df2))
    
    df2["Code"] = strongs_text_repeated[:len(df2)]
    
    dfs.append(df2)
    
   
    # Close the new window
    driver.close()
    
    # Switch back to the main window
    main_window = driver.window_handles[0]
    driver.switch_to.window(main_window)

result = pd.concat(dfs, ignore_index=True)

In [92]:
result

,0,1,2,3,4,Code
0,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,MIS 112.01
1,Department,Statu,Quota,Current,NaN,MIS 112.01
2,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71,NaN,MIS 112.01
3,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1,NaN,MIS 112.01
4,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,MIS 116.01
...,...,...,...,...,...,...
307,Department,Statu,Quota,Current,NaN,MIS 790.12
308,MANAGEMENT INFORMATION SYSTEMS,PHD,Consent Of Instructor,0,NaN,MIS 790.12
309,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,MIS 790.13
310,Department,Statu,Quota,Current,NaN,MIS 790.13


In [93]:
result = result[["Code",0,1,2,3,4]]
result

,Code,0,1,2,3,4
0,MIS 112.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,MIS 112.01,Department,Statu,Quota,Current,NaN
2,MIS 112.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71,NaN
3,MIS 112.01,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1,NaN
4,MIS 116.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
...,...,...,...,...,...,...
307,MIS 790.12,Department,Statu,Quota,Current,NaN
308,MIS 790.12,MANAGEMENT INFORMATION SYSTEMS,PHD,Consent Of Instructor,0,NaN
309,MIS 790.13,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
310,MIS 790.13,Department,Statu,Quota,Current,NaN


In [94]:
result.drop(result.columns[[5]], axis=1, inplace=True)
result

C:\Users\Aykut\AppData\Local\Temp\ipykernel_1216\1594288655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(result.columns[[5]], axis=1, inplace=True)


,Code,0,1,2,3
0,MIS 112.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,MIS 112.01,Department,Statu,Quota,Current
2,MIS 112.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71
3,MIS 112.01,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1
4,MIS 116.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
...,...,...,...,...,...
307,MIS 790.12,Department,Statu,Quota,Current
308,MIS 790.12,MANAGEMENT INFORMATION SYSTEMS,PHD,Consent Of Instructor,0
309,MIS 790.13,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
310,MIS 790.13,Department,Statu,Quota,Current


In [95]:
result.columns = ['Code', 'Department', 'Status', 'Quota', 'Current Quota']
result

,Code,Department,Status,Quota,Current Quota
0,MIS 112.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
1,MIS 112.01,Department,Statu,Quota,Current
2,MIS 112.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71
3,MIS 112.01,MANAGEMENT INFORMATION SYSTEMS,OZELLISANS,2,1
4,MIS 116.01,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
...,...,...,...,...,...
307,MIS 790.12,Department,Statu,Quota,Current
308,MIS 790.12,MANAGEMENT INFORMATION SYSTEMS,PHD,Consent Of Instructor,0
309,MIS 790.13,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:,Departmental Quotas:
310,MIS 790.13,Department,Statu,Quota,Current


In [101]:
words = ["Departmental Quotas:", "Department", "OZELLISANS", "PHD","MASTER","OZELMASTER","OZELPHD"]

result = result[result.Department.isin(words) == False]
result = result[result.Status.isin(words) == False]
result.reset_index(drop=True, inplace=True)
result

,Code,Department,Status,Quota,Current Quota
0,MIS 112.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,71
1,MIS 116.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,74
2,MIS 132.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,87
3,MIS 134.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,60,46
4,MIS 134.02,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,50,38
5,MIS 144.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,104
6,MIS 212.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,91
7,MIS 224.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,101
8,MIS 228.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,107
9,MIS 252.01,MANAGEMENT INFORMATION SYSTEMS,UNDERGRADUATE,Unlimited,99
